In [1]:
from openai import OpenAI
import pathlib
import textwrap
import os
import time
import random
from tqdm.notebook import tqdm
import json
from datetime import datetime

In [2]:
client = OpenAI(
  organization='org-MiSTdq8wllMVuvQ1Es4GrHNX',
  project='proj_rO8d3QjByeL5WcL1nApaiy5B',
)

In [3]:
input_file = "LinearFill"
with open('./data/' + input_file + '.json', 'r') as f:
    data = json.load(f)

In [4]:
def get_json(content_id, prompt):
    sample = {"custom_id": f"{content_id}",
              "method": "POST",
              "url": "/v1/chat/completions",
              "body": {"model": "gpt-4o",
                       "messages": [{"role": "user", 
                                     "content": prompt}],
                       "max_tokens": 1500,
                       "temperature": 1,
                       "top_p": 1,
                       "frequency_penalty":0,
                       "presence_penalty":0}}
    return sample

In [6]:
def definitions():
    contents = []
    for chapter, sections in (data.items()):
        for section, section_definitions in (sections.items()):
            i = 1;
            for q in (section_definitions):
                content_id = (chapter+"|"+section+"|"+str(i))
                i+=1
                dec = random.choice(["true", "false"])
                # message = fr"""Please convert the above definition into a conceptual true or false question with the answer and explanation. The question must test the conceptual understanding of the concept presented, so don't just copy the question and make it trivial. Your explanation must be concise but include an example to illustrate these concepts. Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with three fields, "question", "answer" and "explanation". Definition: {q}. Make the answer {dec} in this case."""
                message = fr"""I will give you a true/false multivariable calculus question. Your job is to transform this question into a new one. If it has numerical values, slightly change them but keep the accuracy correct. You must also give an explanation that is concise but includes the main concepts used. Use examples to illustrate these concepts. Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with three fields, "question", "answer" and "explanation". Question: {q['question'].strip()}. The answer is {q['answer'].strip()}."""
    
                contents.append(get_json(content_id, message))
    return contents
def fill_in_the_blanks():
    contents = []
    for exam, questions in (data.items()):
        for name, q in questions.items():
            i = 1
            for a in q:
                content_id = exam+"|" + name + "|" + str(i)
                i+=1;
                message = fr"""I will give you a question and answer, your job is to create another question similar to this. The new questions should be similar but not the same. Your new question should have different values, structure etc. But all in all should test the same concept. Make sure to be more creative in how you test the concept. 
                Here is the question, Question: {a["question"]}\n Answer: {a["answer"]}. 
                Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with two fields, "question" and "answer."
                """
                contents.append(get_json(content_id, message))
    return contents
    #     for question, section_definitions in (sections.items()):
    #         i = 1;
    #         for q in (section_definitions):
    #             content_id = (chapter+"|"+section+"|"+str(i))
    #             i+=1
    #             dec = random.choice(["true", "false"])
    #             # message = fr"""Please convert the above definition into a conceptual true or false question with the answer and explanation. The question must test the conceptual understanding of the concept presented, so don't just copy the question and make it trivial. Your explanation must be concise but include an example to illustrate these concepts. Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with three fields, "question", "answer" and "explanation". Definition: {q}. Make the answer {dec} in this case."""
    #             message = fr"""I will give you a true/false multivariable calculus question. Your job is to transform this question into a new one. If it has numerical values, slightly change them but keep the accuracy correct. You must also give an explanation that is concise but includes the main concepts used. Use examples to illustrate these concepts. Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with three fields, "question", "answer" and "explanation". Question: {q['question'].strip()}. The answer is {q['answer'].strip()}."""
    # # 
    #             contents.append(get_json(content_id, message))
    # return contents
    

In [7]:
now = datetime.now()
unique_id = now.strftime("%Y%m%d_%H%M")

In [8]:
#contents = definitions()
contents = fill_in_the_blanks()
content_file = input_file + "_" + unique_id
with open(f'batches/batch_files/{content_file}.jsonl', 'w') as f:
    for record in contents:
        json_record = json.dumps(record)
        f.write(json_record + '\n')

In [9]:
from openai import OpenAI

batch_input_file = client.files.create(
  file=open(f"batches/batch_files/{content_file}.jsonl", "rb"),
  purpose="batch"
)

In [10]:
batch_input_file_id = batch_input_file.id
batch_input_file_id

'file-x4vINExPopfIdpOv9rabmtLn'

In [28]:
batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": f"{content_file}"
    }
)
batch_id = batch.id

In [39]:
batch = client.batches.retrieve(batch_id)
batch

Batch(id='batch_6710605b8ca0819087df79f5d6fbdf10', completion_window='24h', created_at=1729126491, endpoint='/v1/chat/completions', input_file_id='file-x4vINExPopfIdpOv9rabmtLn', object='batch', status='cancelling', cancelled_at=None, cancelling_at=1729127359, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1729212891, failed_at=None, finalizing_at=None, in_progress_at=1729126492, metadata={'description': 'LinearFill_20241016_2010'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=107))

In [40]:
batch = client.batches.retrieve("batch_671055e298208190a077a707cf077dad")
batch

Batch(id='batch_671055e298208190a077a707cf077dad', completion_window='24h', created_at=1729123810, endpoint='/v1/chat/completions', input_file_id='file-x4vINExPopfIdpOv9rabmtLn', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1729210210, failed_at=None, finalizing_at=None, in_progress_at=1729123811, metadata={'description': 'LinearFill_20241016_2010'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=107))

In [16]:
output_file_id = batch.output_file_id
content = client.files.content(output_file_id)

In [17]:
with open(f"./batches/batch_responses/{content_file}.jsonl", "w") as f:
    f.write(content.content.decode("utf-8"))

In [19]:
questions = []
with open(f"./batches/batch_responses/{content_file}.jsonl", "r") as file:
    for line in file:
        json_object = json.loads(line)
        custom_id = json_object['custom_id'].split('|')
        q = json_object['response']['body']['choices'][0]["message"]["content"]
        q = q.replace("`", "").replace("json", "").replace("\n", "")
        q = q.replace("\\\\", "\\").replace("\\", "\\\\")
  
        try:
            json_question = json.loads(q)
        except:
            q = q.replace("\\", "\\\\")
            print(q)
            try:
                json_question = json.loads(q)
            except:
                pass
        json_question['chapter'] = custom_id[0]
        json_question['section'] = custom_id[1]
        json_question['number'] = custom_id[2]
        questions.append(json_question)

{  "question": "The equation of the tangent plane to the surface $z = 10 + 3x^2 - y^2$ at $Q(1,2,11)$ is $ \\\\framebox{\\\\strut\\\\hspace{3.5cm}}.$",  "answer": "Set $z = g(x,y) = 10 + 3x^2 - y^2$. Then \\\\begin{align*} g_x &= 6x , \\\\ g_x(1,2) = 6 \\\\ g_y &= -2y , \\\\ g_y(1,2) = -4 \\\\end{align*} Use the formula for the equation to a tangent plane, at a point $Q(x_0,y_0)$, to a surface of the form $z = g(x,y)$. \\\\begin{align*} (z-z_0) &= g_x(x_0,y_0) (x-x_0) + g_y(x_0,y_0) (y-y_0) \\\\ z- 11 &= g_x(1,2)(x-1) + g_y(1,2)(y-2) \\\\ z &= 6(x-1) -4(y-2) \\\\end{align*} The above is sufficient but we can simplify to $z = 6x - 4y + 3$.}


In [23]:
questions[0]

{'question': 'The maximum value of $w(a,b) = 5a + 7b$ subject to $a^2 + 3b^2 = 18$ is located at a point where $a = \\framebox{\\strut\\hspace{1cm}}$, and $b =  \\framebox{\\strut\\hspace{1cm}}$.\\n',
 'answer': 'Set $h = a^2 + 3b^2 - 18 = 0$, and set $\\nabla w = \\mu \\nabla h$. Then\\n\\begin{align*}\\n\\langle 5,7\\rangle &= \\mu \\langle 2a, 6b\\rangle \\n\\end{align*}\\nFrom which we derive:\\n\\begin{align*}\\na &= \\frac{5}{2\\mu} \\quad \\text{and} \\quad b = \\frac{7}{6\\mu}\\n\\end{align*}\\nSubstitute into the constraint equation:\\n\\begin{align*}\\n18 &= \\left(\\frac{5}{2\\mu}\\right)^2 + 3 \\left(\\frac{7}{6\\mu}\\right)^2 \\\\n18\\mu^2 &= \\frac{25}{4} + \\frac{49}{12} \\\\n18\\mu^2 &= \\frac{75+49}{12} \\\\n18\\mu^2 &= \\frac{124}{12} \\\\n\\mu^2 &= \\frac{124}{216} \\\\n\\mu^2 &= \\frac{31}{54} \\\\n\\mu &= \\pm \\sqrt{\\frac{31}{54}} = \\pm \\frac{\\sqrt{31}}{\\sqrt{54}}\\n\\end{align*}\\nSubstitute back to find $a$ and $b$:\\n\\begin{align*}\\na & = \\frac{5}{2 \\t

In [27]:
all_questions =  dict()
for q in questions:
    chapter = q['chapter']
    section = q['section']
    if (chapter not in all_questions):
        all_questions[chapter] = dict()
    if (section not in all_questions[chapter]):
        all_questions[chapter][section] = []
    try:
        all_questions[chapter][section].append({"question" : q['question'], "answer" : q['answer']})
    except:
        continue

In [28]:
with open(f"./generated_files/{content_file}.json", "w") as file:
    json.dump(all_questions, file)